# Convert Label format to YOLO

In [14]:
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
from collections import Counter
import random

In [ ]:
# # Get list of images
# images = [f for f in os.listdir('Dataset/Train/images') if f.endswith('.jpg')]
# # train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

# # Create new train/val directories
# os.makedirs('yolo_dataset/train/images', exist_ok=True)
# os.makedirs('yolo_dataset/train/labels', exist_ok=True)
# os.makedirs('yolo_dataset/val/images', exist_ok=True)
# os.makedirs('yolo_dataset/val/labels', exist_ok=True)

# # Copy files
# for img in images:
#     shutil.copy(f'Dataset/Train/images/{img}', f'yolo_dataset/train/images/{img}')
#     shutil.copy(f'Dataset/Train/labels/{img.replace(".jpg", ".txt")}', f'yolo_dataset/train/labels/{img.replace(".jpg", ".txt")}')
# # for img in val_imgs:
# #     shutil.copy(f'Dataset/Train/images/{img}', f'split_dataset/val/images/{img}')
# #     shutil.copy(f'Dataset/Train/labels/{img.replace(".jpg", ".txt")}', f'split_dataset/val/labels/{img.replace(".jpg", ".txt")}')

In [15]:
def convert_to_yolo(input_dir, output_dir, images_dir):
    os.makedirs(output_dir, exist_ok=True)
    for label_file in os.listdir(input_dir):
        if not label_file.endswith('.txt'):
            continue
        # Get corresponding image to extract dimensions
        img_file = label_file.replace('.txt', '.jpg')
        img_path = os.path.join(images_dir, img_file)
        try:
            img = Image.open(img_path)
            img_width, img_height = img.size
        except FileNotFoundError:
            print(f"Image {img_file} not found, skipping.")
            continue
        
        # Read label file
        with open(os.path.join(input_dir, label_file), 'r') as f:
            lines = f.readlines()
        
        # Convert to YOLO format
        yolo_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                print(f"Invalid format in {label_file}: {line}")
                continue
            class_id, x_min, y_min, x_max, y_max = map(float, parts)
            # Convert to YOLO format
            x_center = (x_min + x_max) / 2 / img_width
            y_center = (y_min + y_max) / 2 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height
            yolo_lines.append(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        
        # Save converted label
        with open(os.path.join(output_dir, label_file), 'w') as f:
            f.write('\n'.join(yolo_lines))

# Convert train and validation labels
convert_to_yolo('Dataset/Train/labels', 'yolo_dataset/train/labels', 'Dataset/Train/images')
# convert_to_yolo('split_dataset/val/labels', 'yolo_dataset/val/labels', 'split_dataset/val/images')

In [45]:
train_class_apple = []
train_class_banana = []
train_class_grapes = []
train_class_orange = []
train_class_pineapple = []
train_class_watermelon = []
train_mix_class = []

labels_dir = 'yolo_dataset/train/labels'
images_dir = 'yolo_dataset/train/images'

new_labels_dir = 'yolo_dataset/new_train/labels'
new_images_dir = 'yolo_dataset/new_train/images'


for label_file in os.listdir(labels_dir):
    class_ids = set()  # Tập hợp các class trong ảnh này
    with open(os.path.join(labels_dir, label_file)) as f:
        for line in f:
            class_id = line.split()[0]
            class_ids.add(class_id)

    if class_ids == {'0'}:
        train_class_apple.append(label_file)
    elif class_ids == {'1'}:
        train_class_banana.append(label_file)
    elif class_ids == {'2'}:
        train_class_grapes.append(label_file)
    elif class_ids == {'3'}:
        train_class_orange.append(label_file)
    elif class_ids == {'4'}:
        train_class_pineapple.append(label_file)
    elif class_ids == {'5'}:
        train_class_watermelon.append(label_file)
    else:
        train_mix_class.append(label_file)
    

In [46]:
print(f"Số lượng ảnh trước khi lọc số lượng class của Apple: {len(train_class_apple)}")
print(f"Số lượng ảnh trước khi lọc số lượng class của Banana: {len(train_class_banana)}")
print(f"Số lượng ảnh trước khi lọc số lượng class của Grapes: {len(train_class_grapes)}")
print(f"Số lượng ảnh trước khi lọc số lượng class của Orange: {len(train_class_orange)}")
print(f"Số lượng ảnh trước khi lọc số lượng class của Pineapple: {len(train_class_pineapple)}")
print(f"Số lượng ảnh trước khi lọc số lượng class của Watermelon: {len(train_class_watermelon)}")
print(f"Số lượng ảnh mix trước khi lọc số lượng: {len(train_mix_class)}")

Số lượng ảnh trước khi lọc số lượng class của Apple: 1521
Số lượng ảnh trước khi lọc số lượng class của Banana: 1188
Số lượng ảnh trước khi lọc số lượng class của Grapes: 1540
Số lượng ảnh trước khi lọc số lượng class của Orange: 1783
Số lượng ảnh trước khi lọc số lượng class của Pineapple: 582
Số lượng ảnh trước khi lọc số lượng class của Watermelon: 750
Số lượng ảnh mix trước khi lọc số lượng: 267


In [47]:
# Đặt seed để kết quả random có thể lặp lại nếu cần
random.seed(42)

train_class_orange = random.sample(train_class_orange, min(800, len(train_class_orange))) # Orange

train_class_banana_test = train_class_banana * 2
train_class_watermelon_test = train_class_watermelon * 2
train_class_pineapple_test = train_class_pineapple * 2
train_mix_class_test = train_mix_class * 2

In [48]:
print(f"Số lượng ảnh sau khi lọc số lượng class của Apple: {len(train_class_apple)}")
print(f"Số lượng ảnh sau khi lọc số lượng class của Banana: {len(train_class_banana_test)}")
print(f"Số lượng ảnh sau khi lọc số lượng class của Grapes: {len(train_class_grapes)}")
print(f"Số lượng ảnh sau khi lọc số lượng class của Orange: {len(train_class_orange)}")
print(f"Số lượng ảnh sau khi lọc số lượng class của Pineapple: {len(train_class_pineapple_test)}")
print(f"Số lượng ảnh sau khi lọc số lượng class của Watermelon: {len(train_class_watermelon_test)}")
print(f"Số lượng ảnh mix sau khi lọc số lượng: {len(train_mix_class_test)}")

Số lượng ảnh sau khi lọc số lượng class của Apple: 1521
Số lượng ảnh sau khi lọc số lượng class của Banana: 2376
Số lượng ảnh sau khi lọc số lượng class của Grapes: 1540
Số lượng ảnh sau khi lọc số lượng class của Orange: 800
Số lượng ảnh sau khi lọc số lượng class của Pineapple: 1164
Số lượng ảnh sau khi lọc số lượng class của Watermelon: 1500
Số lượng ảnh mix sau khi lọc số lượng: 534


In [49]:
all_train_files = (
    train_class_apple +
    train_class_grapes +
    train_class_orange 
)


In [50]:
# all_train_files = (
#     train_class_apple +
#     train_class_banana_test +
#     train_class_grapes +
#     train_class_orange_test +
#     train_class_pineapple_test +
#     train_class_watermelon_test +
#     train_mix_class_test
# )


In [51]:
len(all_train_files)

3861

In [52]:
train_class_counts = Counter()
for label_file in all_train_files:
    with open(os.path.join(labels_dir, label_file)) as f:
        for line in f:
            class_id = line.split()[0]
            train_class_counts[class_id] += 1
print(train_class_counts)

Counter({'2': 6373, '3': 6106, '0': 5827})


In [53]:
import shutil

def copy_labels_images(train_list, labels_dir, images_dir, dest_labels_dir, dest_images_dir):
    os.makedirs(dest_labels_dir, exist_ok=True)
    os.makedirs(dest_images_dir, exist_ok=True)
    for label_file in train_list:
        # Copy labels
        label_src_path = os.path.join(labels_dir, label_file)
        label_dest_path = os.path.join(dest_labels_dir, label_file)
        shutil.copy(label_src_path, label_dest_path)
        
        # Copy images
        image_file = label_file.replace('.txt', '.jpg')
        image_src_path = os.path.join(images_dir, image_file)
        image_dest_path = os.path.join(dest_images_dir, image_file)
        if os.path.exists(image_src_path):
            shutil.copy(image_src_path, image_dest_path)
        else:
            print(f"WARNING: File ảnh không tồn tại: {image_src_path}")
            
            
copy_labels_images(all_train_files, labels_dir, images_dir, new_labels_dir, new_images_dir)

In [54]:
os.makedirs('temp_b', exist_ok=True)
os.makedirs('temp_p', exist_ok=True)
os.makedirs('temp_w', exist_ok=True)
os.makedirs('temp_mix', exist_ok=True)

In [56]:
import shutil

def duplicate_rare_classes(train_list, labels_dir, images_dir, dest_labels_dir, dest_images_dir, duplicate_factor=2):
    os.makedirs(dest_labels_dir, exist_ok=True)
    os.makedirs(dest_images_dir, exist_ok=True)
    train_files = []
    
    for label_file in train_list:
        if not label_file.endswith('.txt'):
            continue
        label_src_path = os.path.join(labels_dir, label_file)
        label_dest_path = os.path.join(dest_labels_dir, label_file)
        image_file = label_file.replace('.txt', '.jpg')
        image_src_path = os.path.join(images_dir, image_file)
        image_dest_path = os.path.join(dest_images_dir, image_file)
        if os.path.exists(image_src_path):
            shutil.copy(label_src_path, label_dest_path)
            shutil.copy(image_src_path, image_dest_path)
            train_files.append(label_file)
        else:
            print(f"WARNING: File ảnh không tồn tại: {image_src_path}")
        
        for i in range(1, duplicate_factor):
            new_label_file = label_file.replace('.txt', f'_copy{i}.txt')
            new_image_file = image_file.replace('.jpg', f'_copy{i}.jpg')
            new_label_dest_path = os.path.join(dest_labels_dir, new_label_file)
            new_image_dest_path = os.path.join(dest_images_dir, new_image_file)
            shutil.copy(label_src_path, new_label_dest_path)
            if os.path.exists(image_src_path):
                shutil.copy(image_src_path, new_image_dest_path)
                train_files.append(new_label_file)
            else:
                print(f"WARNING: File ảnh không tồn tại: {image_src_path}")
    
    return train_files

In [57]:
dest_labels_dir_b = 'temp_b/labels'
dest_labels_dir_p = 'temp_p/labels'
dest_labels_dir_w = 'temp_w/labels'
dest_labels_dir_mix = 'temp_mix/labels'

dest_images_dir_b = 'temp_b/images'
dest_images_dir_p = 'temp_p/images'
dest_images_dir_w = 'temp_w/images'
dest_images_dir_mix = 'temp_mix/images'

duplicate_rare_classes(train_class_banana, labels_dir, images_dir, dest_labels_dir_b, dest_images_dir_b, duplicate_factor=2)
duplicate_rare_classes(train_class_pineapple, labels_dir, images_dir, dest_labels_dir_p, dest_images_dir_p, duplicate_factor=2)
duplicate_rare_classes(train_class_watermelon, labels_dir, images_dir, dest_labels_dir_w, dest_images_dir_w, duplicate_factor=2)
duplicate_rare_classes(train_mix_class, labels_dir, images_dir, dest_labels_dir_mix, dest_images_dir_mix, duplicate_factor=2)

['0074691e66aa85c0_jpg.rf.2fc1e81bbe5377b89f9c912c52ff7905.txt',
 '0074691e66aa85c0_jpg.rf.2fc1e81bbe5377b89f9c912c52ff7905_copy1.txt',
 '00a70c466ad1fe24_jpg.rf.cf298c47a082719e705775c1ec343775.txt',
 '00a70c466ad1fe24_jpg.rf.cf298c47a082719e705775c1ec343775_copy1.txt',
 '00c1d2697deeaea1_jpg.rf.2231307959d5b30b34df054654821be2.txt',
 '00c1d2697deeaea1_jpg.rf.2231307959d5b30b34df054654821be2_copy1.txt',
 '00c1d2697deeaea1_jpg.rf.2390da6ff442cd3a6e8909983b89d2e6.txt',
 '00c1d2697deeaea1_jpg.rf.2390da6ff442cd3a6e8909983b89d2e6_copy1.txt',
 '00fd68005c636fda_jpg.rf.32869db931ff7ac0e41839024f2c7d79.txt',
 '00fd68005c636fda_jpg.rf.32869db931ff7ac0e41839024f2c7d79_copy1.txt',
 '00fd68005c636fda_jpg.rf.a299977af9decdbdf72144569d3265ec.txt',
 '00fd68005c636fda_jpg.rf.a299977af9decdbdf72144569d3265ec_copy1.txt',
 '0125268651daa59c_jpg.rf.1d58f76b6e8efc1435aa7ec133af0e39.txt',
 '0125268651daa59c_jpg.rf.1d58f76b6e8efc1435aa7ec133af0e39_copy1.txt',
 '0125268651daa59c_jpg.rf.9c23d30ac0a681e06d738f

In [58]:
images_b = [f for f in os.listdir('temp_b/images') if f.endswith('.jpg')]
for img in images_b:
    shutil.copy(f'temp_b/images/{img}', f'yolo_dataset/new_train/images/{img}')
    shutil.copy(f'temp_b/labels/{img.replace(".jpg", ".txt")}', f'yolo_dataset/new_train/labels/{img.replace(".jpg", ".txt")}')

images_p = [f for f in os.listdir('temp_p/images') if f.endswith('.jpg')]
for img in images_p:
    shutil.copy(f'temp_p/images/{img}', f'yolo_dataset/new_train/images/{img}')
    shutil.copy(f'temp_p/labels/{img.replace(".jpg", ".txt")}', f'yolo_dataset/new_train/labels/{img.replace(".jpg", ".txt")}')
    
images_w = [f for f in os.listdir('temp_w/images') if f.endswith('.jpg')]
for img in images_w:
    shutil.copy(f'temp_w/images/{img}', f'yolo_dataset/new_train/images/{img}')
    shutil.copy(f'temp_w/labels/{img.replace(".jpg", ".txt")}', f'yolo_dataset/new_train/labels/{img.replace(".jpg", ".txt")}')
    
images_mix = [f for f in os.listdir('temp_mix/images') if f.endswith('.jpg')]
for img in images_mix:
    shutil.copy(f'temp_mix/images/{img}', f'yolo_dataset/new_train/images/{img}')
    shutil.copy(f'temp_mix/labels/{img.replace(".jpg", ".txt")}', f'yolo_dataset/new_train/labels/{img.replace(".jpg", ".txt")}')

In [ ]:
# def duplicate_specific_amount(train_class_banana, labels_dir, images_dir, dest_labels_dir, dest_images_dir, 
#                             current_count=955, target_count=1350):
#     """
#     Duplicate random images from train_class_banana to reach target_count
    
#     Args:
#         train_class_banana: List of label file names (.txt) containing banana class
#         labels_dir: Source labels directory
#         images_dir: Source images directory  
#         dest_labels_dir: Destination labels directory
#         dest_images_dir: Destination images directory
#         current_count: Current number of banana images (955)
#         target_count: Target number of banana images (1350)
#         seed: Random seed for reproducibility
#     """
#     # Set random seed for reproducibility
#     random.seed(42)
    
#     # Create destination directories
#     os.makedirs(dest_labels_dir, exist_ok=True)
#     os.makedirs(dest_images_dir, exist_ok=True)
    
#     # Calculate how many images we need to duplicate
#     needed_duplicates = target_count - current_count
#     print(f"Current: {current_count}, Target: {target_count}")
#     print(f"Need to duplicate: {needed_duplicates} images")
    
#     # First, copy all original files
#     train_files = []
#     valid_files = []
    
#     for label_file in train_class_banana:
#         if not label_file.endswith('.txt'):
#             continue
            
#         label_src_path = os.path.join(labels_dir, label_file)
#         image_file = label_file.replace('.txt', '.jpg')
#         image_src_path = os.path.join(images_dir, image_file)
        
#         # Check if both files exist
#         if os.path.exists(label_src_path) and os.path.exists(image_src_path):
#             # Copy original files
#             label_dest_path = os.path.join(dest_labels_dir, label_file)
#             image_dest_path = os.path.join(dest_images_dir, image_file)
            
#             shutil.copy(label_src_path, label_dest_path)
#             shutil.copy(image_src_path, image_dest_path)
            
#             train_files.append(label_file)
#             valid_files.append(label_file)
#         else:
#             print(f"WARNING: Missing files for {label_file}")
    
#     # Now randomly select files to duplicate
#     if needed_duplicates > 0 and valid_files:
#         # Create list of files to duplicate (can repeat if needed)
#         files_to_duplicate = []
        
#         # If we need more duplicates than available files, we'll duplicate some files multiple times
#         while len(files_to_duplicate) < needed_duplicates:
#             remaining = needed_duplicates - len(files_to_duplicate)
#             if remaining >= len(valid_files):
#                 # Add all files
#                 files_to_duplicate.extend(valid_files)
#             else:
#                 # Add random sample of remaining files
#                 files_to_duplicate.extend(random.sample(valid_files, remaining))
        
#         # Shuffle the list for random order
#         random.shuffle(files_to_duplicate)
        
#         # Keep track of how many times each file has been duplicated
#         duplicate_count = {}
        
#         # Create duplicates
#         for i, label_file in enumerate(files_to_duplicate):
#             # Count duplicates for this file
#             if label_file not in duplicate_count:
#                 duplicate_count[label_file] = 0
#             duplicate_count[label_file] += 1
            
#             # Create new filenames
#             copy_suffix = f"_dup{duplicate_count[label_file]}"
#             new_label_file = label_file.replace('.txt', f'{copy_suffix}.txt')
#             new_image_file = label_file.replace('.txt', f'{copy_suffix}.jpg')
            
#             # Source paths
#             label_src_path = os.path.join(labels_dir, label_file)
#             image_src_path = os.path.join(images_dir, label_file.replace('.txt', '.jpg'))
            
#             # Destination paths
#             new_label_dest_path = os.path.join(dest_labels_dir, new_label_file)
#             new_image_dest_path = os.path.join(dest_images_dir, new_image_file)
            
#             # Copy files
#             shutil.copy(label_src_path, new_label_dest_path)
#             shutil.copy(image_src_path, new_image_dest_path)
            
#             train_files.append(new_label_file)
            
#             if (i + 1) % 50 == 0:
#                 print(f"Duplicated {i + 1}/{needed_duplicates} files...")
                
#     return train_files

In [ ]:
# test_labels_dir = 'temp_b\labels'
# test_images_dir = 'temp_b\images'
# train_files = duplicate_specific_amount(train_class_banana, labels_dir, images_dir, test_labels_dir, test_images_dir, current_count=955, target_count=1350)


Current: 955, Target: 1350
Need to duplicate: 395 images
Duplicated 50/395 files...
Duplicated 100/395 files...
Duplicated 150/395 files...
Duplicated 200/395 files...
Duplicated 250/395 files...
Duplicated 300/395 files...
Duplicated 350/395 files...
